In [3]:
import csv

# Define the paths to the text files
log_txt_files = ["1014-visual1.3-155x86.txt.txt", "1014-visual1.3-155x86.txt.txt.txt", "1014-visual1.3-155x86.txt.txt.txt.txt.txt", "1014-visual1.3-155x86.txt.txt.txt.txt.txt.txt"]  # List all your text files here

def parse_file(file_path, last_value=0):
    """Parse a text file and adjust 'n', 'f', and 'h' events by adding the last_value."""
    csv_data = []
    max_value = last_value

    with open(file_path, "r") as file:
        lines = file.readlines()

    for line in lines:
        line = line.strip()
        if line and line[0].isdigit():  # Only process lines with events
            parts = line.split("\t")
            
            # Ensure each row has enough columns
            while len(parts) < 7:
                parts.append("")

            # Adjust 'n', 'f', or 'h' event values
            if parts[1] in ['n', 'f', 'h']:
                parts[2] = str(int(parts[2]) + last_value)  # Add last 'n/f/h' value
                max_value = max(max_value, int(parts[2]))  # Update max 'n/f/h' value seen
            
            csv_data.append(parts)

    return csv_data, max_value

try:
    # Prepare CSV data with headers
    csv_data = [["Time", "EventType", "x", "y", "z", "r", "n"]]  # CSV headers

    # Initialize the accumulated value
    accumulated_value = 0

    # Loop through each text file and process them
    for txt_file in log_txt_files:
        print(f"Processing {txt_file}...")
        file_data, last_value = parse_file(txt_file, accumulated_value)
        csv_data.extend(file_data)  # Append parsed data
        accumulated_value = last_value  # Update accumulated value for the next file

    # Write combined data to a single CSV file
    csv_filename = "combined_log_data.csv"
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerows(csv_data)

    print(f"CSV file saved as {csv_filename}")

except FileNotFoundError as e:
    print(f"Error: The file {e.filename} was not found.")
except Exception as e:
    print(f"An error occurred: {e}")



Processing 1014-visual1.3-155x86.txt.txt...
Processing 1014-visual1.3-155x86.txt.txt.txt...
Processing 1014-visual1.3-155x86.txt.txt.txt.txt.txt...
Processing 1014-visual1.3-155x86.txt.txt.txt.txt.txt.txt...
CSV file saved as combined_log_data.csv


In [ ]:
import pandas as pd

# Load the CSV file into a dataframe
df_processed = pd.read_csv('combined_1014-visual-1.3.csv', sep=',', header=None, engine='python')

# Naming the columns
df_processed.columns = ['Time', 'EventType', 'Value1', 'Value2', 'Value3', 'Value4', 'Value5']
df_processed

In [22]:
# Preprocess the CSV:
# For events 'n', 'h', and 'f', keep only the event type and trial number (Value1).
# For event 'p', keep all the position and rotation data (x, y, z, and rotation).
def preprocess_event(row):
    event_type = row['EventType']
    if event_type in ['n', 'h', 'f', 's', 'r']:  # Only keep 'Value1' for these events
        return pd.Series([row['Time'], event_type, row['Value1'], None, None, None, None])
    elif event_type == 'p':  # Keep all position and rotation data
        return pd.Series([row['Time'], event_type, None, row['Value1'], row['Value2'], row['Value3'], row['Value4']])
    else:
        return pd.Series([row['Time'], event_type, row['Value1'], row['Value2'], row['Value3'], row['Value4'], row['Value5']])

print("Shape of df_processed after applying preprocess_event:", df_processed.shape)
print("First few rows of df_processed:", df_processed.head(n=10))


Shape of df_processed after applying preprocess_event: (3073469, 7)
First few rows of df_processed:            Time  EventType       Value1 Value2      Value3    Value4 Value5
0          Time  EventType            x      y           z         r      n
1  10:49:58.057          n            1    NaN         NaN       NaN   None
2  10:49:58.057          t            1    0.5  0.03476244       -70     20
3  10:49:58.058          s            0    NaN         NaN       NaN   None
4  10:49:58.171          h            1    NaN         NaN       NaN   None
5  10:49:58.171          r            1    NaN         NaN       NaN   None
6  10:49:58.172          n            2    NaN         NaN       NaN   None
7  10:49:58.172          t            1    0.5   0.1406793       -70     20
8  10:49:58.172          s            0    NaN         NaN       NaN   None
9  10:49:58.192          p  -0.02515763    0.5  0.01511081  90.02986   None


In [23]:
df_processed = df_processed.apply(preprocess_event, axis=1)

In [25]:
# Rename columns for clarity
df_processed.columns = ['Time', 'EventType', 'TrialNumber', 'x', 'y', 'z', 'rotation']

# Save the preprocessed CSV for future use (optional)
df_processed.to_csv('preprocessed_1014-viusal-1.3.csv', index=False)

print("Preprocessing complete, saved as 'preprocessed_1017-viusal-1.41.csv'")

Preprocessing complete, saved as 'preprocessed_data.csv'
